## Scraping of Retsinformation.dk

In [3]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re

In [4]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

HF_TOKEN = os.environ.get("HF_TOKEN")
from huggingface_hub import HfApi, HfFolder

HfFolder.save_token(HF_TOKEN)

## Step 1: Scrape all URLS of vejledninger

In [19]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

def extract_urls_from_page(url):
    """
    Extract all unique URLs from a single search page.

    Args:
    url (str): URL of the page to scrape.

    Returns:
    list: A list of extracted URLs.
    """
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    with webdriver.Chrome(options=options) as driver:
        driver.get(url)
        
        # Wait for the specific element to be loaded
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "document-entry"))
        )
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        search_results = soup.find("div", class_="search-result-list")

        urls = []
        for div in search_results.find_all('div', class_='document-entry'):
            url = div.get('about')
            if url:
                urls.append(url)

    return urls

def make_url_list(n=5, url_prefix='https://www.retsinformation.dk'):
    """
    Function to loop over N pages of search results and extract all URLs.

    Args:
    n (int): Number of pages to scrape.
    url_prefix (str): Prefix to complete the URLs.

    Returns:
    list: A list of all extracted and complete URLs.
    """
    base_url = f'{url_prefix}/documents?dt=180&h=false&page={{page}}&ps=100&r=30'
    all_urls = []

    for page_no in range(0, n):
        full_url = base_url.format(page=page_no)
        page_urls = extract_urls_from_page(full_url)
        complete_urls = [url_prefix + url for url in page_urls]
        all_urls.extend(complete_urls)

    return all_urls

In [22]:
#Took 10 seconds to run
url_list = make_url_list()

#Return first 5 URLs of the list
url_list[0:5]

['https://www.retsinformation.dk/eli/retsinfo/2024/9001',
 'https://www.retsinformation.dk/eli/retsinfo/2024/9000',
 'https://www.retsinformation.dk/eli/retsinfo/2023/10095',
 'https://www.retsinformation.dk/eli/retsinfo/2023/10093',
 'https://www.retsinformation.dk/eli/retsinfo/2023/10092']

## Step 2: Scrape HTML content of vejledninger (bs object)

**OBS**
Nogle vejledninger har tilsynedeladende ikke title i "Titel2" format, men derimod bare "Titel' feks tilfældet for pulverlakering vejledning, andre har class TITLE, og andre igen har ingen men blot font size = 5.... Og et par har slet ingen formatering af overskriften i HTML. 

In [31]:
def extract_title(content_div):
    """
    Extracts the title from a given BeautifulSoup div element.

    This function searches for the title of the content in the provided div element. It checks for various HTML tags and classes to find the title. If none of the specified tags and classes are found, it returns None.

    Args:
        content_div (BeautifulSoup element): The BeautifulSoup element representing a div from which the title is to be extracted.

    Returns:
        str or None: The extracted title as a string, or None if no title is found.
    """
    
    title_elements = [
        {"tag": "p", "class_": "Titel2"},
        {"tag": "p", "class_": "Titel"},
        {"tag": "h1", "class_": "TITLE"},
        {"tag": "font", "attrs": {"size": "5"}}
    ]

    for elem in title_elements:
        if "attrs" in elem:
            title = content_div.find(elem["tag"], **elem["attrs"])
        else:
            title = content_div.find(elem["tag"], class_=elem["class_"])
        
        if title:
            return title.get_text(strip=True).replace("\n", "")

    return None


def scrape_content(urls):
    """
    Scrapes and collects HTML content from a given list of URLs.

    This function navigates to each URL, waits for the page's content to load, and then extracts  HTML content within the "document-content" div using BS. 
    It attempts to identify and use the title of the content key in the resulting dictionary.

    Args:
        urls (list of str): A list of URLs to be scraped.

    Returns:
        dict: A dictionary with titles as keys and the corresponding HTML content as values. If the title can't be determined, the URL is used as the key.
    """
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    
    driver = webdriver.Chrome(options=options)
    result_dict = {}

    for url in urls:
        try:
            driver.get(url)

            # Wait for the specific element to be loaded
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "document-content"))
            )

            # Get the page source and parse it with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, "html.parser")
            content_div = soup.find("div", class_="document-content")

            if content_div:
                title = extract_title(content_div) or str(url)
                result_dict[title] = content_div
            else:
                print("Content not found for URL:", url)

        except Exception as e:
            print(f"An error occurred while processing {url}: {e}")

    # Close the WebDriver
    driver.quit()

    return result_dict

In [30]:
#Took 6 minutes to run
vejledninger_raw = scrape_content(url_list)

In [33]:
#Inspect one of the scraped documents
vejledninger_raw['Arbejde med flyveaske'].get_text(strip=True).replace("\n", "")[0:1000]

'Arbejde med flyveaskeSundhedsfarer og forebyggelse ved arbejde med flyveaske.At-vejledning D.2.21-11. februar - Opdateret juli 2019Erstatter At-meddelelse nr. 4.04.17 af oktober 1990Denne vejledning oplyser om de sundhedsfarer, der er forbundet med arbejde med flyveaske, og om, hvilke foranstaltninger der skal træffes for at imødegå dem.Flyveaske anvendes som fyldmateriale i forbindelse med vejbygning, i cement- og betonindustrien og ved produktion af gasbeton.SundhedsfareFlyveaske består af finkornede partikler, der udskilles af røggasserne fra kulfyrede kraftværker. Flyveaske er et variabelt produkt, hvis egenskaber og kemiske sammensætning afhænger af de anvendte kul, den anvendte forbrændingsteknik og røgrensningsteknikken.Flyveaske indeholder spormængder af bl.a. en række tungmetaller. Indholdet af krystallinsk siliciumdioxid, herunder α-kvarts, kan være over 0,1 pct., hvorfor unge under 18 år ikke må arbejde med flyveaske.Da asken er basisk, kan der ske irritation af hud og slim

## Step 3: Extract and clean text

In [34]:
def extract_text(bs_obj):
    """
    Extracts and concatenates text content from all <p> tags in the provided BeautifulSoup object.

    Args:
        bs_obj (BeautifulSoup): A BeautifulSoup object representing parsed HTML content.

    Returns:
        str: A string containing all text from <p> tags, separated by line breaks.
    """
    paragraphs = bs_obj.find_all("p")
    text = "\n".join(p.get_text(strip=True) for p in paragraphs)
    return text


def clean_text(text):
    """
    Cleans the given text by replacing sequences of two or more line breaks with a double line break.
    Function intended for adding further text cleaning steps if needed.

    Args:
        text (str): The text to be cleaned.

    Returns:
        str: The cleaned text with standardized line breaks.
    """
    import re
    cleaned_text = re.sub(r'\n{2,}', '\n\n', text)
    return cleaned_text


def create_vejl_dict(vejledninger_raw):
    """
    Creates a dictionary of cleaned text for each item in the vejledninger data.

    This function iterates over the vejledninger_raw dictionary, extracts and cleans the text for each item, and stores it in a new dictionary with the same keys.

    Args:
        vejledninger_raw (dict): A dictionary with titles as keys and BeautifulSoup objects as values.

    Returns:
        dict: A dictionary with titles as keys and cleaned text as values.
    """
    vejledninger_dict = {}
    for title, bs_obj in vejledninger_raw.items():
        text = extract_text(bs_obj)
        cleaned_text = clean_text(text)
        vejledninger_dict[title] = cleaned_text
    return vejledninger_dict

In [35]:
vejledninger_text = create_vejl_dict(vejledninger_raw)

In [39]:
vejledning_navn = list(vejledninger_text.keys())

In [40]:
vejledning_navn[0:5]

['Vejledning om regulering af satser fra 1. januar 2024 efter lov om arbejdsskadesikring, lov om sikring mod følger af arbejdsskade, lov om arbejdsskadeforsikring og lov om forsikring mod følger af ulykkestilfælde',
 'Vejledning om satser i 2024 for betaling af udgifter til transport m.v. i forbindelse med lægebehandling, der er begæret af Arbejdsmarkedets Erhvervssikring eller Ankestyrelsen',
 'Vejledning om obligatorisk selvbooking af jobsamtaler for forskellige målgrupper',
 'Vejledning til bekendtgørelse om tilskud til selvstændigt erhvervsdrivende med nedsat arbejdsevne og ansættelse som lønmodtager i fleksjob i en ægtefælles virksomhed',
 'Vejledning om fleksløntilskud m.v.']

In [37]:
url_list[0:5]

['https://www.retsinformation.dk/eli/retsinfo/2024/9001',
 'https://www.retsinformation.dk/eli/retsinfo/2024/9000',
 'https://www.retsinformation.dk/eli/retsinfo/2023/10095',
 'https://www.retsinformation.dk/eli/retsinfo/2023/10093',
 'https://www.retsinformation.dk/eli/retsinfo/2023/10092']

# Pushing to HF

( Loading previous CSV temporarily in order not to have to scrape again)

In [8]:
#Create hugging face dataset using the _from_dict method

from datasets import Dataset
vejledning_navn = list(vejledninger_text.keys())
vejledning_tekst = list(vejledninger_text.values())
vejledning_url = url_list


ds_vejledning = Dataset.from_dict({"vejledning": vejledning_navn, "indhold": vejledning_tekst, "url": vejledning_url})

# add readme
ds_vejledning.info.description = """# Vejledninger fra Retsinformation.dk
Datasættet indeholder alle vejledninger scrapet fra Retsinformation.dk (pr. November 2023).
Datasættet indeholder 2 kolonner: navnet på den givne vejledning (såfremt det fremgik af html'en, ellers fremgår URL) og hele indholdet af vejledningen.
Teksten er renset og formateret således at der er 1 linjeskift (\n) mellem hver sektion ( <p> tag), 
medmindre der er indsat en eller flere tomme sektioner i træk hvormed der i stedet er indsat 2 linjeskift (\n\n)
"""

ds_vejledning.info.dataset_name = "Vejledninger fra Retsinformation.dk"
#ds_vejledning.info.config_name = "retsinformation"

ds_vejledning.info

DatasetInfo(description="# Vejledninger fra Retsinformation.dk\nDatasættet indeholder alle vejledninger scrapet fra Retsinformation.dk (pr. November 2023).\nDatasættet indeholder 2 kolonner: navnet på den givne vejledning (såfremt det fremgik af html'en, ellers fremgår URL) og hele indholdet af vejledningen.\nTeksten er renset og formateret således at der er 1 linjeskift (\n) mellem hver sektion ( <p> tag), \nmedmindre der er indsat en eller flere tomme sektioner i træk hvormed der i stedet er indsat 2 linjeskift (\n\n)\n", citation='', homepage='', license='', features={'vejledning': Value(dtype='string', id=None), 'indhold': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name='Vejledninger fra Retsinformation.dk', config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [9]:
# upload dataset
#Transform to DatasetDict
from datasets import DatasetDict
ds_dict = DatasetDict({"train": ds_vejledning})
ds_dict.push_to_hub(repo_id='dk_retrieval_benchmark', config_name='retsinformation')
#ds_vejledning.push_to_hub(repo_id="dk_retrieval_benchmark", config_name="retsinformation")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]